In [ ]:
# This comes from https://github.com/harterrt/cookiecutter-python-etl/
# Thanks Harter!

In [ ]:
import boto3
import botocore
import os

In [ ]:
repo_dir = "probe-scraper"
output_dir = "/home/hadoop/analyses/probe_data"
cache_dir = "/home/hadoop/analyses/probe_cache"
repo_https_url = "https://github.com/georgf/probe-scraper"

S3_PUBLIC_BUCKET = "telemetry-public-analysis-2"
S3_DATA_PATH = "probe-scraper/data/"
OUTPUT_FILES = ["general.json", "probes.json", "revisions.json"]

In [ ]:
!git clone $repo_https_url $repo_dir

In [ ]:
!cd $repo_dir && git pull origin master && python setup.py bdist_egg

In [ ]:
!mkdir $output_dir && mkdir $cache_dir

In [ ]:
!cd $repo_dir && pip install -r requirements.txt && python probe_scraper/runner.py --outdir $output_dir --tempdir $cache_dir

## Upload the output to S3.

In [ ]:
# Get access to the S3 connect API.
client = boto3.client('s3', 'us-west-2')
transfer = boto3.s3.transfer.S3Transfer(client)

In [ ]:
# Copy the files to S3.
for file_name in OUTPUT_FILES:
    source_path = os.path.join(output_dir, file_name)
    key_path = S3_DATA_PATH + file_name
    print "uploading " + file_name + " to s3: " + key_path
    transfer.upload_file(source_path, S3_PUBLIC_BUCKET, key_path)